# 資料庫應用(三)


## SQLite 操作


In [29]:
# 先裝 mysql的connection driver
# pip install PyMySQL

from sqlalchemy import create_engine

#my_db = create_engine(mysql_db_url)
my_db = create_engine('sqlite:///foo2.db')

- Create

In [30]:
table_name = 'malo_test0903'
sql_cmd_str = "CREATE TABLE IF NOT EXISTS %s(dtime varchar(20) PRIMARY KEY, temp float)" %(table_name)
resultProxy = my_db.execute(sql_cmd_str)
resultProxy.__dict__

{'context': <sqlalchemy.dialects.sqlite.base.SQLiteExecutionContext at 0x1eeaea9fcd0>,
 'dialect': <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite at 0x1eeaea9f610>,
 'cursor': None,
 'cursor_strategy': <sqlalchemy.engine.cursor.NoCursorDMLFetchStrategy at 0x1eead9b3ae0>,
 'connection': <sqlalchemy.engine.base.Connection at 0x1eeaea9f550>,
 '_echo': False,
 '_metadata': <sqlalchemy.engine.cursor._LegacyNoResultMetaData at 0x1eead9fcc10>,
 'rowcount': -1,
 '_soft_closed': True}

- Insert

In [31]:
import datetime
import random

dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(10):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    print(str_dt, temp)

2022-08-01 08:00:00 35.8
2022-08-02 08:00:00 35.0
2022-08-03 08:00:00 36.5
2022-08-04 08:00:00 37.0
2022-08-05 08:00:00 37.3
2022-08-06 08:00:00 36.9
2022-08-07 08:00:00 36.3
2022-08-08 08:00:00 36.5
2022-08-09 08:00:00 37.1
2022-08-10 08:00:00 36.0


In [32]:
# 做insert操作
import datetime
import random

dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(10):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    #print(str_dt, temp)
    sql_cmd_str = "insert into %s (dtime, temp) values('%s', '%s')" %(table_name, str_dt, temp)
    resultProxy=my_db.execute(sql_cmd_str)


In [33]:
# 故意插入重複的
dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(5):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    #print(str_dt, temp)
    sql_cmd_str = "insert into %s (dtime, temp) values('%s', '%s')" %(table_name, str_dt, temp)
    resultProxy=my_db.execute(sql_cmd_str)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: malo_test0903.dtime
[SQL: insert into malo_test0903 (dtime, temp) values('2022-08-01 08:00:00', '35.6')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

- Select

In [34]:
# select
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-01 08:00:00', 36.8), ('2022-08-02 08:00:00', 36.4), ('2022-08-03 08:00:00', 35.7), ('2022-08-04 08:00:00', 35.5), ('2022-08-05 08:00:00', 35.2), ('2022-08-06 08:00:00', 37.0), ('2022-08-07 08:00:00', 36.9), ('2022-08-08 08:00:00', 35.4), ('2022-08-09 08:00:00', 37.3), ('2022-08-10 08:00:00', 36.0)]


- 如果要修改、取代資料

In [35]:
# 做upsert操作
import datetime
import random

dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(15):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    #print(str_dt, temp)
    sql_cmd_str = "insert or replace into %s (dtime, temp) values('%s', '%s')" %(table_name, str_dt, temp)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE temp='%s'" %(temp))
    resultProxy=my_db.execute(sql_cmd_str)


- Select

In [38]:
# select
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-01 08:00:00', 36.6), ('2022-08-02 08:00:00', 35.5), ('2022-08-03 08:00:00', 35.4), ('2022-08-04 08:00:00', 35.5), ('2022-08-05 08:00:00', 36.7), ('2022-08-06 08:00:00', 35.5), ('2022-08-07 08:00:00', 37.2), ('2022-08-08 08:00:00', 36.3), ('2022-08-09 08:00:00', 37.2), ('2022-08-10 08:00:00', 36.6), ('2022-08-11 08:00:00', 36.7), ('2022-08-12 08:00:00', 36.5), ('2022-08-13 08:00:00', 35.8), ('2022-08-14 08:00:00', 37.3), ('2022-08-15 08:00:00', 36.8)]


- Select: 有條件的

In [39]:
# select
sql_cmd_str = "select * from %s where temp>36" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-01 08:00:00', 36.6), ('2022-08-05 08:00:00', 36.7), ('2022-08-07 08:00:00', 37.2), ('2022-08-08 08:00:00', 36.3), ('2022-08-09 08:00:00', 37.2), ('2022-08-10 08:00:00', 36.6), ('2022-08-11 08:00:00', 36.7), ('2022-08-12 08:00:00', 36.5), ('2022-08-14 08:00:00', 37.3), ('2022-08-15 08:00:00', 36.8)]


- Delete

In [40]:
resultProxy=my_db.execute("delete from %s where dtime<'2022-08-05' " %(table_name) )

In [41]:
resultProxy.__dict__

{'context': <sqlalchemy.dialects.sqlite.base.SQLiteExecutionContext at 0x1eeaea82e80>,
 'dialect': <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite at 0x1eeaea9f610>,
 'cursor': None,
 'cursor_strategy': <sqlalchemy.engine.cursor.NoCursorDMLFetchStrategy at 0x1eead9b3ae0>,
 'connection': <sqlalchemy.engine.base.Connection at 0x1eeaea9fe80>,
 '_echo': False,
 '_metadata': <sqlalchemy.engine.cursor._LegacyNoResultMetaData at 0x1eead9fcc10>,
 'rowcount': 4,
 '_soft_closed': True}

- Select

In [42]:
# select
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-05 08:00:00', 36.7), ('2022-08-06 08:00:00', 35.5), ('2022-08-07 08:00:00', 37.2), ('2022-08-08 08:00:00', 36.3), ('2022-08-09 08:00:00', 37.2), ('2022-08-10 08:00:00', 36.6), ('2022-08-11 08:00:00', 36.7), ('2022-08-12 08:00:00', 36.5), ('2022-08-13 08:00:00', 35.8), ('2022-08-14 08:00:00', 37.3), ('2022-08-15 08:00:00', 36.8)]


## 練習題：把高雄空氣品質資料寫入資料庫中

In [43]:
# 更好的寫法!
import csv

with open('2018_kh_data.csv', 'r') as f:
    cr = csv.reader(f)
    data_list = list(cr)
data_list[0:5]

[['Name',
  'Date',
  'TSP',
  'PM10',
  'Chloride',
  'Nitrate',
  'Sulfate',
  'Lead',
  'Dust'],
 ['大社工業區', '2018年1月', '93', '', '', '', '', '0.029', '3.6'],
 ['大寮區公所', '2018年1月', '79', '', '', '', '', '0.042', '1.54'],
 ['大樹監測站', '2018年1月', '61', '', '', '', '', '0.02', '1.18'],
 ['仁武國中', '2018年1月', '79', '', '', '', '', '0.058', '2.68']]

In [16]:
# 請填入你的程式



In [44]:
# 先裝 mysql的connection driver
# pip install PyMySQL

from sqlalchemy import create_engine

my_db = create_engine('sqlite:///foo2.db')

- Create

In [45]:
# 我們把tsp, pm10, lead都記下來
table_name = 'malo_khtsp_0903'
sql_cmd_str = "CREATE TABLE IF NOT EXISTS %s(sid varchar(20), dtime varchar(20), tsp float, pm10 float, lead float, PRIMARY KEY(sid, dtime))" %(table_name)
resultProxy = my_db.execute(sql_cmd_str)
resultProxy.__dict__

{'context': <sqlalchemy.dialects.sqlite.base.SQLiteExecutionContext at 0x1eeaea9f5e0>,
 'dialect': <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite at 0x1eeaea82730>,
 'cursor': None,
 'cursor_strategy': <sqlalchemy.engine.cursor.NoCursorDMLFetchStrategy at 0x1eead9b3ae0>,
 'connection': <sqlalchemy.engine.base.Connection at 0x1eeaec1f9d0>,
 '_echo': False,
 '_metadata': <sqlalchemy.engine.cursor._LegacyNoResultMetaData at 0x1eead9fcc10>,
 'rowcount': -1,
 '_soft_closed': True}

In [46]:
# 做upsert操作
import datetime
import random

for item in data_list[1:]:
    sid = item[0]#['Name']
    dtime = item[1]#['Date']
    tsp = item[2]#['TSP']
    pm10 = item[3]#['PM10']
    lead = item[7]#['Lead']
    sql_cmd_str = "insert or replace into %s (sid, dtime, tsp, pm10, lead) values('%s', '%s', '%s', '%s', '%s')" %(
        table_name, sid, dtime, tsp, pm10, lead)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE tsp='%s', pm10='%s', lead='%s'" %(tsp, pm10, lead))
    resultProxy=my_db.execute(sql_cmd_str)
    print(sql_cmd_str)
    

insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('大社工業區', '2018年1月', '93', '', '0.029')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('大寮區公所', '2018年1月', '79', '', '0.042')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('大樹監測站', '2018年1月', '61', '', '0.02')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('仁武國中', '2018年1月', '79', '', '0.058')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('半屏山電信局', '2018年1月', '104', '67', '0.014')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('左營國小', '2018年1月', '98', '64', '0.006')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('永安工業區', '2018年1月', '83', '', '0.052')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('西子灣中山大學', '2018年1月', '84', '60', '0.043')
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) valu

In [47]:
# 做upsert操作
import datetime
import random

for item in data_list[1:]:
    sid = item[0]#['Name']
    dtime = item[1]#['Date']
    tsp = item[2]#['TSP']
    if tsp=='':
        tsp = 'NULL'
    pm10 = item[3]#['PM10']
    if pm10=='':
        pm10 = 'NULL'
    lead = item[7]#['Lead']
    if lead=='':
        lead = 'NULL'
    sql_cmd_str = "insert or replace into %s (sid, dtime, tsp, pm10, lead) values('%s', '%s', %s, %s, %s)" %(
        table_name, sid, dtime, tsp, pm10, lead)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE tsp=%s, pm10=%s, lead=%s" %(tsp, pm10, lead))
    resultProxy=my_db.execute(sql_cmd_str)
    print(sql_cmd_str)
    

insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('大社工業區', '2018年1月', 93, NULL, 0.029)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('大寮區公所', '2018年1月', 79, NULL, 0.042)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('大樹監測站', '2018年1月', 61, NULL, 0.02)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('仁武國中', '2018年1月', 79, NULL, 0.058)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('半屏山電信局', '2018年1月', 104, 67, 0.014)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('左營國小', '2018年1月', 98, 64, 0.006)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('永安工業區', '2018年1月', 83, NULL, 0.052)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('西子灣中山大學', '2018年1月', 84, 60, 0.043)
insert or replace into malo_khtsp_0903 (sid, dtime, tsp, pm10, lead) values('岡山醫院', '2018年1月', 102, N

In [48]:
# 找出某個站的資料: where
sid = '左營國小'
sql_cmd_str = "select * from %s where sid='%s'" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('左營國小', '2018年10月', 89.0, 51.0, 0.024), ('左營國小', '2018年11月', 65.0, 38.0, 0.035), ('左營國小', '2018年12月', 90.0, 52.0, 0.019), ('左營國小', '2018年1月', 98.0, 64.0, 0.006), ('左營國小', '2018年2月', 59.0, 32.0, 0.017), ('左營國小', '2018年3月', 54.0, 36.0, 0.068), ('左營國小', '2018年4月', 61.0, 33.0, 0.028), ('左營國小', '2018年5月', 49.0, 38.0, 0.047), ('左營國小', '2018年6月', 22.0, 16.0, 0.034), ('左營國小', '2018年7月', 37.0, 16.0, 0.022), ('左營國小', '2018年8月', 22.0, 14.0, 0.004), ('左營國小', '2018年9月', 32.0, 22.0, 0.02)]


In [49]:
# 找出某個站的資料, 並排序: where, order
sid = '仁武國中'
sql_cmd_str = "select * from %s where sid='%s' order by dtime DESC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('仁武國中', '2018年9月', 46.0, None, 0.006), ('仁武國中', '2018年8月', 19.0, None, -0.009), ('仁武國中', '2018年7月', 56.0, None, 0.02), ('仁武國中', '2018年6月', 48.0, None, 0.034), ('仁武國中', '2018年5月', 44.0, None, 0.058), ('仁武國中', '2018年4月', 78.0, None, 0.035), ('仁武國中', '2018年3月', 108.0, None, 0.052), ('仁武國中', '2018年2月', 86.0, None, 0.033), ('仁武國中', '2018年1月', 79.0, None, 0.058), ('仁武國中', '2018年12月', 134.0, None, 0.018), ('仁武國中', '2018年11月', 115.0, None, 0.1), ('仁武國中', '2018年10月', 91.0, None, 0.042)]


In [51]:
# 找出某個站的資料, 並排序: where, order
sid = '仁武國中'
sql_cmd_str = "select * from %s where sid='%s' order by dtime ASC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('仁武國中', '2018年10月', 91.0, None, 0.042), ('仁武國中', '2018年11月', 115.0, None, 0.1), ('仁武國中', '2018年12月', 134.0, None, 0.018), ('仁武國中', '2018年1月', 79.0, None, 0.058), ('仁武國中', '2018年2月', 86.0, None, 0.033), ('仁武國中', '2018年3月', 108.0, None, 0.052), ('仁武國中', '2018年4月', 78.0, None, 0.035), ('仁武國中', '2018年5月', 44.0, None, 0.058), ('仁武國中', '2018年6月', 48.0, None, 0.034), ('仁武國中', '2018年7月', 56.0, None, 0.02), ('仁武國中', '2018年8月', 19.0, None, -0.009), ('仁武國中', '2018年9月', 46.0, None, 0.006)]


----
### 為何排序出了問題!?

In [52]:
# 先把資料都刪掉: DELETE FROM table_name
sql_cmd_str = "DELETE FROM %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)


In [53]:
# 確認資料
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
data

[]

In [54]:
# 做upsert操作: 試著改時間格式
import datetime
import random

for item in data_list[1:]:
    sid = item[0]#['Name']
    dtime = item[1]#['Date']
    dtime = dtime.replace('月', '')
    dtime_tmp = dtime.split('年')
    dtime = '%s-%02d' %(dtime_tmp[0], int(dtime_tmp[1]))
    tsp = item[2]#['TSP']
    if tsp=='':
        tsp = 'NULL'
    pm10 = item[3]#['PM10']
    if pm10=='':
        pm10 = 'NULL'
    lead = item[7]#['Lead']
    if lead=='':
        lead = 'NULL'
    sql_cmd_str = "insert or replace into %s (sid, dtime, tsp, pm10, lead) values('%s', '%s', %s, %s, %s)" %(
        table_name, sid, dtime, tsp, pm10, lead)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE tsp=%s, pm10=%s, lead=%s" %(tsp, pm10, lead))
    resultProxy=my_db.execute(sql_cmd_str)
    #print(sql_cmd_str)
    

In [55]:
# 這樣的時間格式未來如何轉換
dt1 = datetime.datetime.strptime('2018-03','%Y-%m') #把時間字串轉為datetime物件
dt1

datetime.datetime(2018, 3, 1, 0, 0)

- 再來取資料，並做排序

In [56]:
# 找出某個站的資料, 並排序: where, order
sid = '仁武國中'
sql_cmd_str = "select * from %s where sid='%s' order by dtime ASC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('仁武國中', '2018-01', 79.0, None, 0.058), ('仁武國中', '2018-02', 86.0, None, 0.033), ('仁武國中', '2018-03', 108.0, None, 0.052), ('仁武國中', '2018-04', 78.0, None, 0.035), ('仁武國中', '2018-05', 44.0, None, 0.058), ('仁武國中', '2018-06', 48.0, None, 0.034), ('仁武國中', '2018-07', 56.0, None, 0.02), ('仁武國中', '2018-08', 19.0, None, -0.009), ('仁武國中', '2018-09', 46.0, None, 0.006), ('仁武國中', '2018-10', 91.0, None, 0.042), ('仁武國中', '2018-11', 115.0, None, 0.1), ('仁武國中', '2018-12', 134.0, None, 0.018)]


In [57]:
# 找出某個站的資料, 並排序: where, order
sid = '大寮區公所'
sql_cmd_str = "select * from %s where sid='%s' order by dtime ASC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('大寮區公所', '2018-01', 79.0, None, 0.042), ('大寮區公所', '2018-02', 91.0, None, 0.018), ('大寮區公所', '2018-03', 92.0, None, 0.046), ('大寮區公所', '2018-04', 75.0, None, 0.062), ('大寮區公所', '2018-05', 43.0, None, 0.021), ('大寮區公所', '2018-06', 41.0, None, 0.006), ('大寮區公所', '2018-07', 44.0, None, 0.013), ('大寮區公所', '2018-08', 14.0, None, 0.01), ('大寮區公所', '2018-09', 31.0, None, 0.043), ('大寮區公所', '2018-10', 79.0, None, 0.041), ('大寮區公所', '2018-11', 98.0, None, 0.016), ('大寮區公所', '2018-12', 124.0, None, 0.012)]
